In [15]:
from binance import Client
import pandas as pd
import configparser
import os
from datetime import datetime
import dateutil.relativedelta


config = configparser.ConfigParser()
config.read_file(open('config.cfg'))
PUBLIC_KEY = config.get('BINANCE', 'PUBLIC_KEY')
PRIVATE_KEY = config.get('BINANCE', 'PRIVATE_KEY')

if not os.path.exists('Last_Updated_Socket.txt'):
    last_updated_time = str(
        datetime.now() + dateutil.relativedelta.relativedelta(months=-2))
else:
    with open('Last_Updated_Socket.txt', 'r') as f:
        last_updated_time = f.read()

client = Client(PUBLIC_KEY, PRIVATE_KEY)
df = pd.DataFrame()

for coins in client.get_all_tickers():
    if((coins["symbol"] == "FISBUSD")):
        historical_data = client.get_historical_klines(
            symbol=coins["symbol"], interval=Client.KLINE_INTERVAL_1DAY, start_str="2 Nov,2022")
        coin_df = pd.DataFrame(historical_data, columns=['Open Time', 'Open', 'High', 'Low', 'Close', 'Volume','Close Time', 'Quote Asset Volume', 'Number of Trades', 'TB Base Volume', 'TB Quote Volume', 'Ignore'])
        coin_df['Open']=coin_df['Open'].apply(lambda x: float(x))
        coin_df['High']=coin_df['High'].apply(lambda x: float(x))
        coin_df['Low']=coin_df['Low'].apply(lambda x: float(x))
        coin_df['Close']=coin_df['Close'].apply(lambda x: float(x))
        coin_df['Open Time'] = pd.to_datetime(coin_df['Open Time']/1000, unit='s')
        coin_df['Close Time'] = pd.to_datetime(coin_df['Close Time']/1000, unit='s')

coin_df.to_csv("STOCK.csv",index=False)
